<a href="https://colab.research.google.com/github/abou-zithar/ZEP_task_Retail__shipping__goods_to_USA/blob/main/ZEP_task_Retail__shipping__goods_usa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import lib
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# connect to drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# use pandas to read data 
filepath='/content/drive/MyDrive/datasets/ZepDataSet.xlsx'
data=pd.read_excel(filepath,parse_dates=['location_arrival_date','location_departure_date','origin_departure_date','train_arrival_date'])

In [ ]:
data.head()

,tripuid,origin_location,destination_location,sighting_location,sighting_location_lat,sighting_location_lon,location_arrival_date,location_departure_date,origin_departure_date,train_arrival_date
0,10054656400104001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","STERLING HEIGHTS, MI",42.537908,-83.037650,2021-04-30 19:28:00,2021-04-30 19:28:00,2021-04-30 19:28:00,2021-05-03 12:21:00
1,10054656400104001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","WARREN, MI",42.499210,-83.037413,2021-04-30 21:45:00,2021-04-30 21:46:00,2021-04-30 19:28:00,2021-05-03 12:21:00
2,10054656400104001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","DETROIT STERLING, MI",42.572998,-83.040002,2021-05-01 00:00:00,2021-05-01 12:30:00,2021-04-30 19:28:00,2021-05-03 12:21:00
3,10054656400104001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","DETROIT NORTH YARD, MI",42.406106,-83.037049,2021-05-01 13:30:00,2021-05-01 15:30:00,2021-04-30 19:28:00,2021-05-03 12:21:00
4,10054656400104001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","OAKWOOD, MI",42.278053,-83.170834,2021-05-01 18:17:00,2021-05-01 18:17:00,2021-04-30 19:28:00,2021-05-03 12:21:00


In [ ]:
data.tail()

,tripuid,origin_location,destination_location,sighting_location,sighting_location_lat,sighting_location_lon,location_arrival_date,location_departure_date,origin_departure_date,train_arrival_date
1605,10063453900023001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","CLAPPER, MO",39.588213,-91.845766,2020-08-13 04:24:00,2020-08-13 04:24:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1606,10063453900023001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","MOBERLY, MO",39.419797,-92.436711,2020-08-13 05:23:00,2020-08-13 05:43:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1607,10063453900023001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","CLIFTON HILL, MO",39.438175,-92.665850,2020-08-13 07:01:00,2020-08-13 07:01:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1608,10063453900023001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","MISSOURI CITY, MO",39.223788,-94.321008,2020-08-13 12:25:00,2020-08-13 12:25:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1609,10063453900023001315859000566790000,"STEHEIGHT, MI","VOLTZ, MO","VOLTZ, MO",39.186341,-94.429775,2020-08-13 13:24:00,2020-08-13 16:30:00,2020-08-11 04:24:00,2020-08-13 13:24:00


In [ ]:
data.drop(['tripuid'],inplace=True,axis=1)

In [ ]:
data['origin_location'].unique()

array(['STEHEIGHT, MI'], dtype=object)

In [ ]:
data['destination_location'].unique()

array(['VOLTZ, MO'], dtype=object)

In [ ]:
data.drop(['destination_location','origin_location'],inplace=True,axis=1)

In [ ]:
data.duplicated().sum()

2

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.head()

,sighting_location,sighting_location_lat,sighting_location_lon,location_arrival_date,location_departure_date,origin_departure_date,train_arrival_date
0,"STERLING HEIGHTS, MI",42.537908,-83.037650,2021-04-30 19:28:00,2021-04-30 19:28:00,2021-04-30 19:28:00,2021-05-03 12:21:00
1,"WARREN, MI",42.499210,-83.037413,2021-04-30 21:45:00,2021-04-30 21:46:00,2021-04-30 19:28:00,2021-05-03 12:21:00
2,"DETROIT STERLING, MI",42.572998,-83.040002,2021-05-01 00:00:00,2021-05-01 12:30:00,2021-04-30 19:28:00,2021-05-03 12:21:00
3,"DETROIT NORTH YARD, MI",42.406106,-83.037049,2021-05-01 13:30:00,2021-05-01 15:30:00,2021-04-30 19:28:00,2021-05-03 12:21:00
4,"OAKWOOD, MI",42.278053,-83.170834,2021-05-01 18:17:00,2021-05-01 18:17:00,2021-04-30 19:28:00,2021-05-03 12:21:00


In [ ]:
data.tail()

,sighting_location,sighting_location_lat,sighting_location_lon,location_arrival_date,location_departure_date,origin_departure_date,train_arrival_date
1605,"CLAPPER, MO",39.588213,-91.845766,2020-08-13 04:24:00,2020-08-13 04:24:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1606,"MOBERLY, MO",39.419797,-92.436711,2020-08-13 05:23:00,2020-08-13 05:43:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1607,"CLIFTON HILL, MO",39.438175,-92.665850,2020-08-13 07:01:00,2020-08-13 07:01:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1608,"MISSOURI CITY, MO",39.223788,-94.321008,2020-08-13 12:25:00,2020-08-13 12:25:00,2020-08-11 04:24:00,2020-08-13 13:24:00
1609,"VOLTZ, MO",39.186341,-94.429775,2020-08-13 13:24:00,2020-08-13 16:30:00,2020-08-11 04:24:00,2020-08-13 13:24:00


In [ ]:
data.shape

(1608, 7)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1608 entries, 0 to 1609
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sighting_location        1608 non-null   object        
 1   sighting_location_lat    1608 non-null   float64       
 2   sighting_location_lon    1608 non-null   float64       
 3   location_arrival_date    1608 non-null   datetime64[ns]
 4   location_departure_date  1608 non-null   datetime64[ns]
 5   origin_departure_date    1608 non-null   datetime64[ns]
 6   train_arrival_date       1608 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(2), object(1)
memory usage: 100.5+ KB


<h4> as we can see the data has no nulls and the data is datatime64 type 

In [ ]:
def getDeltaOfTiem(feature_from,feature_to):
  data['Trip_time'] = (data[feature_from] - data[feature_to])

  # create a column with timedelta as total hours, as a float type
  data['tot_hour_diff'] = (data[feature_from] - data[feature_to]) / pd.Timedelta(hours=1)

  # create a colume with timedelta as total minutes, as a float type
  data['tot_mins_diff'] = (data[feature_from] - data[feature_to]) / pd.Timedelta(minutes=1)

getDeltaOfTiem('train_arrival_date','origin_departure_date')

In [ ]:
data['Trip_time']

KeyError: ignored

In [ ]:
def recreateTimeAndDateFeatures(Feature):
  data['Y_'+Feature] = [d.year for d in data[Feature]]
  data['M_'+Feature] = [d.month for d in data[Feature]]
  data['D_'+Feature] = [d.day for d in data[Feature]]
  data['H_'+Feature] = [d.hour for d in data[Feature]]
  data['M_'+Feature] = [d.minute for d in data[Feature]]



recreateTimeAndDateFeatures('location_arrival_date')
recreateTimeAndDateFeatures('location_departure_date')
recreateTimeAndDateFeatures('origin_departure_date')
recreateTimeAndDateFeatures('train_arrival_date')
data.drop(['location_arrival_date','location_departure_date','origin_departure_date','train_arrival_date'],inplace=True,axis=1)

In [ ]:
data.head()

In [ ]:
data.info()

<h1> Visualization⌛

In [ ]:
import seaborn as sns

In [ ]:

sns.displot(data=data, x="Y_origin_departure_date", kind="kde")

In [ ]:

sns.displot(data=data, x="D_origin_departure_date")
sns.displot(data=data, x="D_train_arrival_date")


In [ ]:
sns.histplot(data=data,x='H_train_arrival_date',y='D_train_arrival_date')

In [ ]:
sns.displot(data=data, x="H_origin_departure_date")
sns.displot(data=data, x="H_train_arrival_date")

In [ ]:
sns.displot(data=data, x="M_origin_departure_date")
sns.displot(data=data, x="M_train_arrival_date")